# Préparation des données et ingénierie pour la création d'un modèle de prédiction de changement de voie dangereux à l'aide du jeu de données NGSIM

Ce code vise à préparer les données et effectuer l'ingénierie nécessaire pour créer un modèle de prédiction visant à déterminer si un changement de voie est dangereux ou non. Il utilise le jeu de données NGSIM, qui contient des informations sur les trajectoires de véhicules.

In [1]:
import pandas as pd
import numpy as np
import sys
from tqdm import tqdm
from pandarallel import pandarallel
import warnings

# premiere code de preparation

Ceci est un script qui vise à trouver et ajouter les voisins de chaque ligne de données dans le DataFrame "new_df". Il utilise la fonction "get_neighbors" pour trouver les voisins en fonction des critères de distance et de voie spécifiés. Les résultats sont ensuite ajoutés au DataFrame "new_df". Le script utilise également la bibliothèque "pandarallel" pour paralléliser le calcul et accélérer le processus. Les avertissements sont ignorés avec warnings.filterwarnings('ignore'), et une barre de progression est affichée avec "tqdm" pour suivre l'avancement du traitement des véhicules. Enfin, les données préparées sont sauvegardées dans un fichier de sortie spécifié.

In [ ]:

warnings.filterwarnings('ignore')
pandarallel.initialize(progress_bar=True, nb_workers=4)


# script to find and add neighbours for each row of data

# Fonction pour obtenir les voisins de chaque ligne de données
def get_neighbors(row,data,max_dist=70,max_width=5.4):
    import pandas as pd
    
    # Extraire les informations de la ligne actuelle
    timestamp = row['Global_Time']
    ego_lane = row['Lane_ID']
    ego_y = row['Local_Y']
    ego_x = row['Local_X']
    ego_length = row['v_Length']
    ego_width = row['v_Width']
    
    # Calculer les distances latérales et longitudinales entre les véhicules
    lat_dist1 = (data['Local_X']-ego_x)*0.305
    lat_dist2 = (ego_x-data['Local_X'])*0.305 # entre les ligne
    long_dist = abs(ego_y-data['Local_Y'])*0.305  # dans la meme ligne
    
    # Conditions pour trouver les véhicules voisins dans la même voie (à droite et à gauche)
    r_Lane_cond = (data['Global_Time']==timestamp) & (lat_dist1<=max_width) & (long_dist<=max_dist)
    l_Lane_cond = (data['Global_Time']==timestamp) & (lat_dist2<=max_width) & (long_dist<=max_dist)
    #features=['Vehicle_ID','Local_X','Local_Y','v_Vel','v_Acc','Lane_ID']
    
    # Sélectionner les véhicules voisins dans la même voie (à droite et à gauche)
    r_vehicles = data[r_Lane_cond]
    r_vehicles.sort_values(by=['Local_Y'], inplace=True)
    l_vehicles = data[l_Lane_cond]
    l_vehicles.sort_values(by=['Local_Y'], inplace=True)
    
    # Extraire les informations des véhicules voisins
    try:
        r_vehicles_id = r_vehicles['Vehicle_ID'].values
        r_vehicles_X = r_vehicles['Local_X'].values
        r_vehicles_Y = r_vehicles['Local_Y'].values
        r_vehicles_vel = r_vehicles['v_Vel'].values
        r_vehicles_acc = r_vehicles['v_Acc'].values
        r_vehicles_lane = r_vehicles['Lane_ID'].values
    except:
        r_v_id = 0
        r_vehicles_X = 0
        r_vehicles_Y = 0
        r_vehicles_vel = 0
        r_vehicles_acc = 0
        r_vehicles_lane = 0
    try:
        l_vehicles_id = l_vehicles['Vehicle_ID'].values
        l_vehicles_X = l_vehicles['Local_X'].values
        l_vehicles_Y = l_vehicles['Local_Y'].values
        l_vehicles_vel = l_vehicles['v_Vel'].values
        l_vehicles_acc = l_vehicles['v_Acc'].values
        l_vehicles_lane = l_vehicles['Lane_ID'].values
    except:
        l_v_id = 0
        l_vehicles_X = 0
        l_vehicles_Y = 0
        l_vehicles_vel = 0
        l_vehicles_acc = 0
        l_vehicles_lane=0
        
    # Retourner les informations des véhicules voisins sous forme de série pandas
    return pd.Series([r_vehicles_id ,r_vehicles_X, r_vehicles_Y,r_vehicles_lane, r_vehicles_vel, r_vehicles_acc, l_vehicles_id, 
                      l_vehicles_X, l_vehicles_Y, l_vehicles_lane,l_vehicles_vel, l_vehicles_acc])



def prepare_data(inputfile, output, outputdir):
    pd.set_option('mode.chained_assignment', None) # Désactiver les avertissements pour les assignations en chaîne
    data = pd.read_csv(inputfile) # Charger les données à partir du fichier d'entrée
    vehicle_ids = data.groupby('Vehicle_ID').groups.keys() # Obtenir les identifiants uniques des véhicules
    new_df = pd.DataFrame() # Créer un DataFrame vide pour stocker les données préparées
    progress = 0
    
    # Boucle sur chaque identifiant de véhicule
    with tqdm(total=len(vehicle_ids)) as pbar:
        for v_id in vehicle_ids:
                v_df = data[data.Vehicle_ID==v_id] # Extraire les données du véhicule actuel
                others = data[data.Vehicle_ID!=v_id] # Extraire les données des autres véhicules
                
                 # Appliquer la fonction get_neighbors en parallèle pour trouver les voisins du véhicule actuel
                v_df[['Right_v_IDs','Right_v_X','Right_v_Y','Right_v_Lanes','Right_v_Vels','Right_v_Acc','Left_v_IDs','Left_v_X',
                      'Left_v_Y','Left_v_Lanes', 'Left_v_Vels','Left_v_Acc']] = v_df.parallel_apply(get_neighbors,axis=1,args=(others,))
            
                    
                new_df = new_df.append(v_df) # Ajouter les données préparées du véhicule actuel au DataFrame global
                new_df.to_csv(output,index=False) # Sauvegarder les données préparées dans un fichier de sortie
                progress = progress+1
                sys.stdout.write("\r%d of %d vehicles" % (progress, len(vehicle_ids))) # Afficher la progression
                sys.stdout.flush()
                pbar.update(1)
    return new_df

# Chemins des fichiers d'entrée et de sortie
filename1='C:/Users/Admin/Desktop/BDSAS S2/DATA MINING 2/NGSIM I80/trajectories-0400-0415.csv'
outputdir1='C:/Users/Admin/Desktop/BDSAS S2/DATA MINING 2/NGSIM I80/Data/prepared/'
outputfile1='C:/Users/Admin/Desktop/BDSAS S2/DATA MINING 2/NGSIM I80/Data/prepared/trajectories-prepared.csv'
prepare_data(filename1,outputfile1,outputdir1) # Appeler la fonction pour préparer les données

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


  0%|                                                                                         | 0/1725 [00:00<?, ?it/s]

1 of 1725 vehicles

  0%|                                                                              | 1/1725 [00:36<17:37:32, 36.81s/it]

2 of 1725 vehicles

  0%|                                                                              | 2/1725 [00:57<13:07:12, 27.41s/it]

In [ ]:
# Chargez les données préparées
new_df = pd.read_csv('C:/Users/Admin/Desktop/BDSAS S2/DATA MINING 2/NGSIM I80/Data/prepared/trajectories-prepared.csv')

Transforme les champ de type series (pandas.core.series.Series) vers des liste

In [5]:
#fonction pour Transforme les champ de type series (pandas.core.series.Series) vers des liste
def Series_to_liste(data , feauters,spl="") :
  for j in range(len(data)) :
      for i in feauters :
          if  str(data[i][j]) == '[]' :
              data[i][j]=[]
          else :
              data[i][j]=[float(x) for x in str(data[i][j][1:-1]).split(spl)]

In [ ]:
feauters=['Right_v_IDs', 'Right_v_X','Right_v_Y', 'Right_v_Lanes', 'Right_v_Vels', 'Right_v_Acc', 'Left_v_IDs', 'Left_v_X', 'Left_v_Y','Left_v_Lanes', 'Left_v_Vels', 'Left_v_Acc']
Series_to_liste(new_df , feauters,spl="")

# ajouter 'Right_cuts', 'Left_cuts'

Dans ce code, la fonction detect_cuts prend une ligne de la DataFrame new_df en entrée, extrait les informations sur les véhicules voisins à droite et à gauche, et retourne une liste de coupes potentielles pour chaque côté.et le résultat est stocké dans deux nouvelles colonnes 'Right_cuts' et 'Left_cuts' de la DataFrame. Enfin, la DataFrame mise à jour est enregistrée dans un fichier CSV 'trajectories_with_cuts.csv'.

In [ ]:
def detect_cuts(row):
    """
    Fonction pour détecter les coupes potentielles des véhicules voisins pour une ligne de la DataFrame
    """
    r_vehicles_id = row['Right_v_IDs']
    r_vehicles_X = row['Right_v_X']
    r_vehicles_Y = row['Right_v_Y']
    r_vehicles_lane = row['Right_v_Lanes']
    r_vehicles_vel = row['Right_v_Vels']
    r_vehicles_acc = row['Right_v_Acc']
    l_vehicles_id = row['Left_v_IDs']
    l_vehicles_X = row['Left_v_X']
    l_vehicles_Y = row['Left_v_Y']
    l_vehicles_lane = row['Left_v_Lanes']
    l_vehicles_vel = row['Left_v_Vels']
    l_vehicles_acc = row['Left_v_Acc']
    
    # Détection des coupes potentielles pour les véhicules à droite
    right_cuts = []
    for i in range(len(r_vehicles_id)):
        print ("nice"+str(r_vehicles_id))
        if (r_vehicles_lane[i] == row['Lane_ID']) and (r_vehicles_Y[i] > row['Local_Y']):
            if (r_vehicles_X[i] > row['Local_X']) and (r_vehicles_vel[i] > row['v_Vel']):
                right_cuts.append(r_vehicles_id[i])
                
                          
    
    # Détection des coupes potentielles pour les véhicules à gauche
    left_cuts = []
    for i in range(len(l_vehicles_id)):
        print ("nice"+str(l_vehicles_id))
        if (l_vehicles_lane[i] == row['Lane_ID']) and (l_vehicles_Y[i] > row['Local_Y']):
            if (l_vehicles_X[i] < row['Local_X']) and (l_vehicles_vel[i] > row['v_Vel']):
                left_cuts.append(l_vehicles_id[i])
    
    return pd.Series([right_cuts, left_cuts])


# Appliquer la fonction detect_cuts sur chaque ligne de la DataFrame new_df
new_df[['Right_cuts', 'Left_cuts']] = new_df.apply(detect_cuts, axis=1)#
# La fonction apply est utilisée pour appliquer cette fonction à chaque ligne de la DataFrame.

# Enregistrer le résultat dans un fichier CSV
new_df.to_csv('/content/drive/MyDrive/dataset/trajectories_with_cuts.csv', index=False)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
nice[166.0, 186.0, 175.0, 158.0, 183.0, 185.0, 176.0, 143.0, 173.0, 169.0, 179.0, 159.0, 174.0, 163.0, 162.0, 161.0, 168.0, 149.0, 125.0, 156.0, 164.0, 150.0, 165.0, 120.0, 142.0, 146.0, 140.0, 144.0, 126.0, 151.0, 117.0, 136.0]
nice[166.0, 186.0, 175.0, 158.0, 183.0, 185.0, 176.0, 143.0, 173.0, 169.0, 179.0, 159.0, 174.0, 163.0, 162.0, 161.0, 168.0, 149.0, 125.0, 156.0, 164.0, 150.0, 165.0, 120.0, 142.0, 146.0, 140.0, 144.0, 126.0, 151.0, 117.0, 136.0]
nice[166.0, 186.0, 175.0, 158.0, 183.0, 185.0, 176.0, 143.0, 173.0, 169.0, 179.0, 159.0, 174.0, 163.0, 162.0, 161.0, 168.0, 149.0, 125.0, 156.0, 164.0, 150.0, 165.0, 120.0, 142.0, 146.0, 140.0, 144.0, 126.0, 151.0, 117.0, 136.0]
nice[166.0, 158.0, 143.0, 169.0, 174.0, 163.0, 156.0, 125.0, 120.0, 142.0, 144.0, 117.0]
nice[166.0, 158.0, 143.0, 169.0, 174.0, 163.0, 156.0, 125.0, 120.0, 142.0, 144.0, 117.0]
nice[166.0, 158.0, 143.0, 169.0, 174.0, 163.0, 156.0, 125

In [ ]:
new_df

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,...,Right_v_Vels,Right_v_Acc,Left_v_IDs,Left_v_X,Left_v_Y,Left_v_Lanes,Left_v_Vels,Left_v_Acc,Right_cuts,Left_cuts
0,1,12,884,1113433136100,16.884,48.213,6042842.116,2133117.662,14.3,6.4,...,[11.16],[10.67],[36.0],[27.492],[84.274],[3.0],[11.16],[10.67],[],[]
1,1,13,884,1113433136200,16.938,49.463,6042842.012,2133118.909,14.3,6.4,...,[11.99],[6.76],[36.0],[27.433],[85.465],[3.0],[11.99],[6.76],[],[]
2,1,14,884,1113433136300,16.991,50.712,6042841.908,2133120.155,14.3,6.4,...,[12.45],[1.49],[36.0],[27.423],[86.736],[3.0],[12.45],[1.49],[],[]
3,1,15,884,1113433136400,17.045,51.963,6042841.805,2133121.402,14.3,6.4,...,[12.53],[-0.3],[36.0],[27.434],[88.003],[3.0],[12.53],[-0.3],[],[]
4,1,16,884,1113433136500,17.098,53.213,6042841.701,2133122.649,14.3,6.4,...,[12.51],[-0.16],[36.0],[27.434],[89.254],[3.0],[12.51],[-0.16],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69588,187,1025,700,1113433237400,17.571,761.576,6042754.589,2133825.708,12.8,6.3,...,"[35.24, 25.0, 8.18, 44.72, 34.95, 48.1, 32.56]","[-2.78, 0.0, -11.2, -0.12, 1.19, -2.18, 11.2]","[182.0, 186.0, 176.0, 175.0, 183.0, 173.0, 185...","[4.73, 58.083, 39.85, 52.001, 60.648, 41.167, ...","[724.29, 750.741, 770.613, 802.444, 809.346, 8...","[1.0, 5.0, 4.0, 5.0, 6.0, 4.0, 5.0, 3.0, 6.0, ...","[35.24, 24.33, 29.53, 30.0, 15.99, 35.02, 19.0...","[-2.78, 1.33, 6.16, 0.0, 0.0, 0.31, 0.0, 0.0, ...",[],[163.0]
69589,187,1026,700,1113433237500,17.578,763.865,6042754.302,2133827.963,12.8,6.3,...,"[35.03, 25.0, 8.26, 44.72, 35.02, 47.87]","[-1.41, 0.0, 11.2, 0.43, 0.13, -3.18]","[182.0, 186.0, 176.0, 175.0, 183.0, 173.0, 185...","[4.711, 57.467, 39.86, 52.021, 60.463, 41.192,...","[727.791, 753.136, 773.628, 805.443, 810.945, ...","[1.0, 5.0, 4.0, 5.0, 6.0, 4.0, 5.0, 3.0, 6.0, ...","[35.03, 24.8, 29.95, 30.0, 15.99, 35.06, 19.0,...","[-1.41, 8.09, 1.68, 0.0, 0.0, 0.85, 0.0, 0.0, ...",[],[163.0]
69590,187,1027,700,1113433237600,17.587,766.239,6042754.002,2133830.322,12.8,6.3,...,"[34.94, 25.0, 13.08, 44.78, 35.03]","[-0.29, 0.0, 11.2, 0.67, -0.2]","[182.0, 186.0, 176.0, 175.0, 183.0, 173.0, 185...","[4.719, 57.052, 39.87, 52.041, 60.279, 41.215,...","[731.279, 755.639, 776.649, 808.443, 812.543, ...","[1.0, 5.0, 4.0, 5.0, 5.0, 4.0, 5.0, 3.0, 6.0, ...","[34.94, 25.78, 30.05, 30.0, 15.99, 35.21, 18.8...","[-0.29, 11.2, -0.38, 0.0, 0.0, 2.09, -3.08, 0....",[],[163.0]
69591,187,1028,700,1113433237700,17.595,768.703,6042753.690,2133832.774,12.8,6.3,...,"[34.93, 25.0, 20.18, 44.86, 35.01]","[0.27, 0.0, 11.2, 0.75, -0.16]","[182.0, 186.0, 176.0, 175.0, 183.0, 173.0, 185...","[4.726, 56.816, 39.881, 52.06, 60.093, 41.229,...","[734.767, 758.27, 779.652, 811.443, 814.142, 8...","[1.0, 5.0, 4.0, 5.0, 5.0, 4.0, 5.0, 3.0, 6.0, ...","[34.93, 26.91, 30.02, 30.0, 15.99, 35.64, 18.3...","[0.27, 11.18, -0.28, 0.0, 0.0, 5.63, -5.78, 0....",[],[163.0]


In [ ]:
# Chargez les données préparées avec les deux nouvelles colonnes 'Right_cuts' et 'Left_cuts'
df = pd.read_csv('C:/Users/Admin/Desktop/BDSAS S2/DATA MINING 2/NGSIM I80/Data/prepared/trajectories_with_cuts.csv')

# La fonction describe_cut_in_R_L

La fonction describe_cut_in_R_L permet de décrire les coupes (cut-ins) à droite et à gauche dans un DataFrame donné. Voici comment elle fonctionne :

 - Elle initialise les compteurs s, d, s1, d1, z, et z1 à zéro. Ces compteurs sont utilisés pour compter différents types de coupes.
 - Les listes f1 et f2 contiennent les noms des colonnes correspondant aux coupes à droite et à gauche respectivement.
 - La fonction parcourt chaque ligne du DataFrame df et compte le nombre de lignes où les colonnes de coupes à droite et à gauche sont vides ou non vides.
 - Les compteurs s, d, s1, et d1 sont mis à jour en conséquence.
 - De plus, la fonction compte également le nombre de lignes où à la fois les coupes à droite et à gauche sont non vides (représenté par le compteur z) et le nombre de lignes où à la fois les coupes à droite et à gauche sont vides (représenté par le compteur z1).
 - Enfin, la fonction affiche les résultats, indiquant le nombre de coupes à droite, le nombre de non-coupes à droite, le nombre de coupes à gauche, le nombre de non-coupes à gauche, le nombre de coupes à la fois à droite et à gauche, et le nombre de non-coupes à la fois à droite et à gauche.


In [ ]:
#descreption de cut-in
def describe_cut_in_R_L(df) :
  s=0
  d=0
  s1=0
  d1=0
  z=0
  z1=0
  f1=['Right_cuts']
  f2=['Left_cuts']
  for j in range(len(df)) :
    for i in f1 :
      if str(df[i][j]) == '[]' :
        s=s+1
      else :
        d=d+1
    for i in f2 :
      if str(df[i][j]) == '[]' :
        s1=s1+1
      else :
        d1=d1+1
  
    if str(df['Right_cuts'][j]) !='[]' and str(df['Left_cuts'][j]) != '[]' :
        z=z+1
    if str(df['Right_cuts'][j]) =='[]' and str(df['Left_cuts'][j]) == '[]' :
        z1=z1+1
  print ("N° no cut_in R : " + str(s))
  print ("N° cut_in R : " + str(d))
  print ("N° no cut_in L : " + str(s1))
  print ("N° cut_in L : " + str(d1))
  print ("N° cut-in L and R : " + str(z))
  print ("N° no cut-in L and R : " + str(z1))
    

describe_cut_in_R_L(df) 

N° no cut_in R : 40648
N° cut_in R : 28945
N° no cut_in L : 41408
N° cut_in L : 28185
N° cut-in L and R : 12929
N° no cut-in L and R : 25392


In [ ]:
#transforme  'Right_cuts','Left_cuts'  vers list
f=['Right_cuts','Left_cuts']
split=","
Series_to_liste(df,f,split)

<ipython-input-83-4b1eeeeec8b9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i][j]=[]
<ipython-input-83-4b1eeeeec8b9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i][j]=[float(x) for x in str(data[i][j][1:-1]).split(spl)]


In [ ]:
df

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,...,Right_v_Vels,Right_v_Acc,Left_v_IDs,Left_v_X,Left_v_Y,Left_v_Lanes,Left_v_Vels,Left_v_Acc,Right_cuts,Left_cuts
0,1,12,884,1113433136100,16.884,48.213,6042842.116,2133117.662,14.3,6.4,...,[11.16],[10.67],[36.0],[27.492],[84.274],[3.0],[11.16],[10.67],[],[]
1,1,13,884,1113433136200,16.938,49.463,6042842.012,2133118.909,14.3,6.4,...,[11.99],[6.76],[36.0],[27.433],[85.465],[3.0],[11.99],[6.76],[],[]
2,1,14,884,1113433136300,16.991,50.712,6042841.908,2133120.155,14.3,6.4,...,[12.45],[1.49],[36.0],[27.423],[86.736],[3.0],[12.45],[1.49],[],[]
3,1,15,884,1113433136400,17.045,51.963,6042841.805,2133121.402,14.3,6.4,...,[12.53],[-0.3],[36.0],[27.434],[88.003],[3.0],[12.53],[-0.3],[],[]
4,1,16,884,1113433136500,17.098,53.213,6042841.701,2133122.649,14.3,6.4,...,[12.51],[-0.16],[36.0],[27.434],[89.254],[3.0],[12.51],[-0.16],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69588,187,1025,700,1113433237400,17.571,761.576,6042754.589,2133825.708,12.8,6.3,...,"[35.24, 25.0, 8.18, 44.72, 34.95, 48.1, 32.56]","[-2.78, 0.0, -11.2, -0.12, 1.19, -2.18, 11.2]","[182.0, 186.0, 176.0, 175.0, 183.0, 173.0, 185...","[4.73, 58.083, 39.85, 52.001, 60.648, 41.167, ...","[724.29, 750.741, 770.613, 802.444, 809.346, 8...","[1.0, 5.0, 4.0, 5.0, 6.0, 4.0, 5.0, 3.0, 6.0, ...","[35.24, 24.33, 29.53, 30.0, 15.99, 35.02, 19.0...","[-2.78, 1.33, 6.16, 0.0, 0.0, 0.31, 0.0, 0.0, ...",[],[]
69589,187,1026,700,1113433237500,17.578,763.865,6042754.302,2133827.963,12.8,6.3,...,"[35.03, 25.0, 8.26, 44.72, 35.02, 47.87]","[-1.41, 0.0, 11.2, 0.43, 0.13, -3.18]","[182.0, 186.0, 176.0, 175.0, 183.0, 173.0, 185...","[4.711, 57.467, 39.86, 52.021, 60.463, 41.192,...","[727.791, 753.136, 773.628, 805.443, 810.945, ...","[1.0, 5.0, 4.0, 5.0, 6.0, 4.0, 5.0, 3.0, 6.0, ...","[35.03, 24.8, 29.95, 30.0, 15.99, 35.06, 19.0,...","[-1.41, 8.09, 1.68, 0.0, 0.0, 0.85, 0.0, 0.0, ...",[],[]
69590,187,1027,700,1113433237600,17.587,766.239,6042754.002,2133830.322,12.8,6.3,...,"[34.94, 25.0, 13.08, 44.78, 35.03]","[-0.29, 0.0, 11.2, 0.67, -0.2]","[182.0, 186.0, 176.0, 175.0, 183.0, 173.0, 185...","[4.719, 57.052, 39.87, 52.041, 60.279, 41.215,...","[731.279, 755.639, 776.649, 808.443, 812.543, ...","[1.0, 5.0, 4.0, 5.0, 5.0, 4.0, 5.0, 3.0, 6.0, ...","[34.94, 25.78, 30.05, 30.0, 15.99, 35.21, 18.8...","[-0.29, 11.2, -0.38, 0.0, 0.0, 2.09, -3.08, 0....",[],[]
69591,187,1028,700,1113433237700,17.595,768.703,6042753.690,2133832.774,12.8,6.3,...,"[34.93, 25.0, 20.18, 44.86, 35.01]","[0.27, 0.0, 11.2, 0.75, -0.16]","[182.0, 186.0, 176.0, 175.0, 183.0, 173.0, 185...","[4.726, 56.816, 39.881, 52.06, 60.093, 41.229,...","[734.767, 758.27, 779.652, 811.443, 814.142, 8...","[1.0, 5.0, 4.0, 5.0, 5.0, 4.0, 5.0, 3.0, 6.0, ...","[34.93, 26.91, 30.02, 30.0, 15.99, 35.64, 18.3...","[0.27, 11.18, -0.28, 0.0, 0.0, 5.63, -5.78, 0....",[],[]


# Fonction pour trouver l'id du véhicule cut le plus proche

La fonction find_nearest_cut permet de trouver le véhicule coupant le plus proche pour chaque ligne du DataFrame df. Voici comment elle fonctionne :

 - Elle récupère les listes de coupes à droite (right_cuts) et à gauche (left_cuts) de la ligne courante.
 - Ensuite, elle concatène ces deux listes pour former une liste all_cuts contenant tous les identifiants des véhicules coupants.
 - Si la liste all_cuts est vide, la fonction renvoie NaN, indiquant qu'aucun véhicule ne coupe la trajectoire pour cette ligne.
 - Sinon, elle itère sur chaque identifiant de coupe dans la liste all_cuts et calcule la distance euclidienne entre la position du véhicule courant et la position du véhicule coupant correspondant.
 - Les distances sont stockées dans la liste distances.
 - En utilisant np.argmin(distances), la fonction trouve l'indice de la distance minimale, ce qui correspond à l'indice du cut le plus proche.
 - Finalement, la fonction renvoie l'identifiant du cut le plus proche (nearest_cut_id).
 - La fonction find_nearest_cut est appliquée à chaque ligne du DataFrame df en utilisant df.apply(find_nearest_cut, axis=1), et le résultat est enregistré dans une colonne "Target".
 - Si aucun véhicule ne coupe la trajectoire, la valeur de la colonne "Target" sera NaN.
 - Le DataFrame mis à jour est enregistré dans un fichier CSV à l'emplacement spécifié.

In [ ]:

def find_nearest_cut(row):
    right_cuts = row['Right_cuts']
    left_cuts = row['Left_cuts']
    
    # Concaténer les listes de cuts
    all_cuts = right_cuts + left_cuts
    
    if len(all_cuts) == 0:
        return np.nan
    
    # Trouver la distance entre le véhicule courant et chaque cut
    distances = []
    for cut_id in all_cuts:
        cut_row = df.loc[new_df['Vehicle_ID'] == cut_id].iloc[0]
        distance = ((cut_row['Local_X'] - row['Local_X'])**2 + (cut_row['Local_Y'] - row['Local_Y'])**2)**0.5
        distances.append(distance)
    
    # Trouver l'id du cut le plus proche
    nearest_cut_id = all_cuts[np.argmin(distances)]
    
    return nearest_cut_id

# Appliquer la fonction find_nearest_cut sur chaque ligne de la DataFrame new_df
df['Target'] = df.apply(find_nearest_cut, axis=1)
# Si aucun véhicule ne coupe la trajectoire, la fonction renvoie NaN.
# Enregistrer le résultat dans un fichier CSV
df.to_csv('/content/drive/MyDrive/dataset/trajectories_with_cuts.csv', index=False)

In [ ]:
df = pd.read_csv('C:/Users/Admin/Desktop/BDSAS S2/DATA MINING 2/NGSIM I80/Data/prepared/trajectories_with_cuts_and_target.csv')

# Ajouter un champ Cut_in

cette colonne dépend de la valeur de la colonne "Target", si la valeur de la colonne "Target" est vide (représentée par une chaîne vide " "). Si c'est le cas, la fonction renvoie 0, sinon elle renvoie 1

In [ ]:
# Ajouter un champ Cut_in
df['Cut_in'] = df['Target'].apply(lambda x: 0 if pd.isna(x) else 1)

In [ ]:
df

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,...,Left_v_IDs,Left_v_X,Left_v_Y,Left_v_Lanes,Left_v_Vels,Left_v_Acc,Right_cuts,Left_cuts,Target,Cut_in
0,1,12,884,1113433136100,16.884,48.213,6042842.116,2133117.662,14.3,6.4,...,[36.0],[27.492],[84.274],[3.0],[11.16],[10.67],[],[],NaN,0
1,1,13,884,1113433136200,16.938,49.463,6042842.012,2133118.909,14.3,6.4,...,[36.0],[27.433],[85.465],[3.0],[11.99],[6.76],[],[],NaN,0
2,1,14,884,1113433136300,16.991,50.712,6042841.908,2133120.155,14.3,6.4,...,[36.0],[27.423],[86.736],[3.0],[12.45],[1.49],[],[],NaN,0
3,1,15,884,1113433136400,17.045,51.963,6042841.805,2133121.402,14.3,6.4,...,[36.0],[27.434],[88.003],[3.0],[12.53],[-0.3],[],[],NaN,0
4,1,16,884,1113433136500,17.098,53.213,6042841.701,2133122.649,14.3,6.4,...,[36.0],[27.434],[89.254],[3.0],[12.51],[-0.16],[],[],NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69588,187,1025,700,1113433237400,17.571,761.576,6042754.589,2133825.708,12.8,6.3,...,"[182.0, 186.0, 176.0, 175.0, 183.0, 173.0, 185...","[4.73, 58.083, 39.85, 52.001, 60.648, 41.167, ...","[724.29, 750.741, 770.613, 802.444, 809.346, 8...","[1.0, 5.0, 4.0, 5.0, 6.0, 4.0, 5.0, 3.0, 6.0, ...","[35.24, 24.33, 29.53, 30.0, 15.99, 35.02, 19.0...","[-2.78, 1.33, 6.16, 0.0, 0.0, 0.31, 0.0, 0.0, ...",[],[163.0],163.0,1
69589,187,1026,700,1113433237500,17.578,763.865,6042754.302,2133827.963,12.8,6.3,...,"[182.0, 186.0, 176.0, 175.0, 183.0, 173.0, 185...","[4.711, 57.467, 39.86, 52.021, 60.463, 41.192,...","[727.791, 753.136, 773.628, 805.443, 810.945, ...","[1.0, 5.0, 4.0, 5.0, 6.0, 4.0, 5.0, 3.0, 6.0, ...","[35.03, 24.8, 29.95, 30.0, 15.99, 35.06, 19.0,...","[-1.41, 8.09, 1.68, 0.0, 0.0, 0.85, 0.0, 0.0, ...",[],[163.0],163.0,1
69590,187,1027,700,1113433237600,17.587,766.239,6042754.002,2133830.322,12.8,6.3,...,"[182.0, 186.0, 176.0, 175.0, 183.0, 173.0, 185...","[4.719, 57.052, 39.87, 52.041, 60.279, 41.215,...","[731.279, 755.639, 776.649, 808.443, 812.543, ...","[1.0, 5.0, 4.0, 5.0, 5.0, 4.0, 5.0, 3.0, 6.0, ...","[34.94, 25.78, 30.05, 30.0, 15.99, 35.21, 18.8...","[-0.29, 11.2, -0.38, 0.0, 0.0, 2.09, -3.08, 0....",[],[163.0],163.0,1
69591,187,1028,700,1113433237700,17.595,768.703,6042753.690,2133832.774,12.8,6.3,...,"[182.0, 186.0, 176.0, 175.0, 183.0, 173.0, 185...","[4.726, 56.816, 39.881, 52.06, 60.093, 41.229,...","[734.767, 758.27, 779.652, 811.443, 814.142, 8...","[1.0, 5.0, 4.0, 5.0, 5.0, 4.0, 5.0, 3.0, 6.0, ...","[34.93, 26.91, 30.02, 30.0, 15.99, 35.64, 18.3...","[0.27, 11.18, -0.28, 0.0, 0.0, 5.63, -5.78, 0....",[],[163.0],163.0,1


In [ ]:
new_df=df

In [ ]:
#transforme  'Right_cuts','Left_cuts'  vers list
f=['Right_cuts','Left_cuts']
split=","
Series_to_liste(new_df,f,split)

<ipython-input-13-4b1eeeeec8b9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i][j]=[]
<ipython-input-13-4b1eeeeec8b9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i][j]=[float(x) for x in str(data[i][j][1:-1]).split(spl)]


In [ ]:

#transforme  'Right_v_X','Right_v_Y','Left_v_X','Left_v_Y'  vers list
f=['Right_v_X','Right_v_Y','Left_v_X','Left_v_Y']
split=","
Series_to_liste(new_df,f,split)

<ipython-input-13-4b1eeeeec8b9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i][j]=[float(x) for x in str(data[i][j][1:-1]).split(spl)]
<ipython-input-13-4b1eeeeec8b9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i][j]=[]


In [ ]:
#convertit la série Local_X en valeurs flottantes :
new_df['Local_X'] = new_df['Local_X'].astype(float)
new_df['Local_Y'] = new_df['Local_Y'].astype(float)

In [ ]:
# Enregistrer le résultat dans un fichier CSV
new_df.to_csv('C:/Users/Admin/Desktop/BDSAS S2/DATA MINING 2/NGSIM I80/Data/prepared/trajectories_with_cuts_and_target1.csv', index=False)

In [5]:
new_df = pd.read_csv('C:/Users/Admin/Desktop/BDSAS S2/DATA MINING 2/NGSIM I80/Data/prepared/trajectories_with_cuts_and_target1.csv')

In [7]:
new_df.columns

Index(['Vehicle_ID', 'Frame_ID', 'Total_Frames', 'Global_Time', 'Local_X',
       'Local_Y', 'Global_X', 'Global_Y', 'v_Length', 'v_Width', 'v_Class',
       'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding', 'Following', 'Space_Headway',
       'Time_Headway', 'Right_v_IDs', 'Right_v_X', 'Right_v_Y',
       'Right_v_Lanes', 'Right_v_Vels', 'Right_v_Acc', 'Left_v_IDs',
       'Left_v_X', 'Left_v_Y', 'Left_v_Lanes', 'Left_v_Vels', 'Left_v_Acc',
       'Right_cuts', 'Left_cuts', 'Target', 'Cut_in'],
      dtype='object')

# Fonction pour déterminer si le cut-in est dangereux ou non ( bases sur 7 seconds et reaction_time et Distance de sécurité)

Voici une fonction is_dangerous qui détermine si un changement de voie est dangereux ou non. Voici comment elle fonctionne :

 - Elle utilise les informations de la ligne actuelle pour calculer la distance de sécurité requise, en fonction de la vitesse du véhicule hôte et du temps de réaction du conducteur. 
 - Ensuite, elle recherche les coupes (véhicules se rapprochant de la voie du véhicule actuel) dans les 7 secondes précédant le changement de voie. 
 - Si le véhicule cible (véhicule avec lequel il y a un risque de collision) est présent parmi les coupes, la fonction vérifie la distance entre le véhicule hôte et le véhicule coupant. 
 - Si la distance est inférieure à la distance de sécurité, la fonction retourne 1 (dangereux), sinon elle retourne 0 (non dangereux). 
 - Si aucun véhicule coupant n'est trouvé dans les 7 secondes précédant le changement de voie, la fonction retourne également 0. 
 - Enfin, la fonction est appliquée à chaque ligne du DataFrame new_df, et le résultat est ajouté dans une nouvelle colonne appelée "Dangere".


In [ ]:
def is_dangerous(row):
    """
    Fonction pour déterminer si le cut-in est dangereux ou non
    """
    # Temps de réaction du conducteur (en secondes)
    reaction_time = 1.5 
    
    # Vitesse de la voiture hôte (en m/s)
    v_host = row['v_Vel'] / 3.6
    
    # Distance de sécurité (en mètres)
    safety_distance = v_host * 3 + reaction_time * v_host
    
    # Recherche des voitures coupantes dans les 7 secondes précédant le cut-in
    cuts = []
    cut_in_frame = row['Frame_ID']
    for i, cut_row in new_df.iterrows():
        print ("row_index : " +str(i))  
        cut_frame = cut_row['Frame_ID']
        frame_diff = cut_in_frame - cut_frame
        if frame_diff >= 0 and frame_diff <= 70: #70 FRAME IN 7 SECONS  (1 frame in 0,1 Second)
            cuts += cut_row['Left_cuts']
            cuts += cut_row['Right_cuts']
        
    if row['Target'] in cuts:
        # La voiture coupante est présente dans les 7 secondes précédant le cut-in
        if row['Target'] in row['Left_cuts']:
            x_cut = row['Left_cuts_X'][row['Left_cuts'].index(row['Target'])]
            y_cut = row['Left_cuts_Y'][row['Left_cuts'].index(row['Target'])]
        else:
            x_cut = row['Right_cuts_X'][row['Right_cuts'].index(row['Target'])]
            y_cut = row['Right_cuts_Y'][row['Right_cuts'].index(row['Target'])]
        
        # Calcul de la distance entre la voiture hôte et la voiture coupante
        d = np.sqrt((row['Local_X'] - x_cut)**2 + (row['Local_Y'] - y_cut)**2)
    
        # Vérification si la distance entre la voiture hôte et la voiture coupante est inférieure à la distance de sécurité
        if d < safety_distance:
            return 1 # Dangereux
        else:
            return 0 # Non dangereux
    else:
        # Pas de voiture coupante dans les 7 secondes précédant le cut-in
        return 0
    
# Ajout du champ "Dangere" à la DataFrame
new_df['Dangere'] = new_df.apply(is_dangerous, axis=1)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
row_index : 47219
row_index : 47220
row_index : 47221
row_index : 47222
row_index : 47223
row_index : 47224
row_index : 47225
row_index : 47226
row_index : 47227
row_index : 47228
row_index : 47229
row_index : 47230
row_index : 47231
row_index : 47232
row_index : 47233
row_index : 47234
row_index : 47235
row_index : 47236
row_index : 47237
row_index : 47238
row_index : 47239
row_index : 47240
row_index : 47241
row_index : 47242
row_index : 47243
row_index : 47244
row_index : 47245
row_index : 47246
row_index : 47247
row_index : 47248
row_index : 47249
row_index : 47250
row_index : 47251
row_index : 47252
row_index : 47253
row_index : 47254
row_index : 47255
row_index : 47256
row_index : 47257
row_index : 47258
row_index : 47259
row_index : 47260
row_index : 47261
row_index : 47262
row_index : 47263
row_index : 47264
row_index : 47265
row_index : 47266
row_index : 47267
row_index : 47268
row_index : 47269
row_

In [ ]:
# Enregistrer le résultat dans un fichier CSV
new_df.to_csv('C:/Users/Admin/Desktop/BDSAS S2/DATA MINING 2/NGSIM I80/Data/prepared/trajectories_Dangere_final.csv', index=False)

In [3]:
new_df = pd.read_csv('C:/Users/Admin/Desktop/BDSAS S2/DATA MINING 2/NGSIM I80/Data/prepared/trajectories_Dangere_final.csv')

In [4]:
#nombre de cut_in  dangere

dangerous_cut_ins = new_df[new_df['Dangere'] == 1].groupby('Vehicle_ID')['Cut_in'].sum()
print(len(dangerous_cut_ins))

47


In [5]:
#nombre de cut_in non dangere
dangerous_cut_ins = new_df[new_df['Dangere'] == 0].groupby('Vehicle_ID')['Cut_in'].sum()
print(len(dangerous_cut_ins))

103


# Suppression des colonnes non importantes dans le DataFrame new_df

In [8]:
# Supprimer la colonne "Vehicle_ID"
new_df=new_df.drop(["Vehicle_ID"],axis=1)
# Supprimer la colonne "Right_cuts"
new_df=new_df.drop(["Right_cuts"],axis=1)
# Supprimer la colonne "Left_cuts"
new_df=new_df.drop(["Left_cuts"],axis=1)
# Supprimer plusieurs colonnes simultanément
new_df=new_df.drop(['Right_v_IDs', 'Right_v_X','Right_v_Y', 'Right_v_Lanes', 'Right_v_Vels', 'Right_v_Acc', 'Left_v_IDs', 'Left_v_X', 'Left_v_Y','Left_v_Lanes', 'Left_v_Vels', 'Left_v_Acc'], axis=1)

In [9]:
new_df['Target'] = np.where(new_df['Target'].isna(), 0, new_df['Target'])

In [12]:
#supprime les lignes avec les valeurs manquantes
new_df=new_df.dropna()
new_df

,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID,Preceding,Following,Space_Headway,Time_Headway,Target,Cut_in,Dangere
0,12,884,1113433136100,16.884,48.213,6042842.116,2133117.662,14.3,6.4,2,12.50,0.00,2,0,0,0.00,0.00,0.0,0,0
1,13,884,1113433136200,16.938,49.463,6042842.012,2133118.909,14.3,6.4,2,12.50,0.00,2,0,0,0.00,0.00,0.0,0,0
2,14,884,1113433136300,16.991,50.712,6042841.908,2133120.155,14.3,6.4,2,12.50,0.00,2,0,0,0.00,0.00,0.0,0,0
3,15,884,1113433136400,17.045,51.963,6042841.805,2133121.402,14.3,6.4,2,12.50,0.00,2,0,0,0.00,0.00,0.0,0,0
4,16,884,1113433136500,17.098,53.213,6042841.701,2133122.649,14.3,6.4,2,12.50,0.00,2,0,0,0.00,0.00,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69587,1024,700,1113433237300,17.563,759.321,6042754.870,2133823.500,12.8,6.3,2,22.50,2.60,2,169,190,103.78,4.61,163.0,1,0
69588,1025,700,1113433237400,17.571,761.576,6042754.589,2133825.708,12.8,6.3,2,22.87,4.19,2,169,190,102.68,4.49,163.0,1,0
69589,1026,700,1113433237500,17.578,763.865,6042754.302,2133827.963,12.8,6.3,2,23.42,7.04,2,169,190,101.08,4.32,163.0,1,0
69590,1027,700,1113433237600,17.587,766.239,6042754.002,2133830.322,12.8,6.3,2,24.10,7.28,2,169,190,99.88,4.14,163.0,1,0


In [13]:
# Enregistrer le résultat dans un fichier CSV   final
new_df.to_csv('C:/Users/Admin/Desktop/BDSAS S2/DATA MINING 2/NGSIM I80/Data/prepared/trajectories_final.csv', index=False)

# **les modeles**




Ce code utilise plusieurs modèles de classification tels que Logistic Regression (LR), K-nearest neighbors (KNN), Classification And Regression Trees (CART), Support Vector Machine (SVM), Random Forests (RF) et Gradient Boosting Machine (GBM). Chaque modèle est intégré dans un pipeline comprenant une étape de prétraitement des données avec StandardScaler() pour la mise à l'échelle des données, suivi du modèle correspondant. Les données sont ensuite divisées en ensembles d'entraînement et de test à l'aide de train_test_split(). Chaque modèle est entraîné sur l'ensemble d'entraînement, utilisé pour prédire les valeurs de l'ensemble de test, puis évalué en utilisant la métrique de précision (accuracy) avec accuracy_score().

In [14]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
#from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Diviser les données en variables d'entraînement et variable cible
X = new_df.drop(['Dangere'], axis=1)
y = new_df['Dangere']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer un pipeline de prétraitement et de modèle pour chaque modèle
pipelines = [
    ('CART', Pipeline([('scaler', StandardScaler()), ('model', DecisionTreeClassifier())])),
    ('RF', Pipeline([('scaler', StandardScaler()), ('model', RandomForestClassifier())])),
    ('KNN', Pipeline([('scaler', StandardScaler()), ('model', KNeighborsClassifier())])),
    ('LightGBM', Pipeline([('scaler', StandardScaler()), ('model', LGBMClassifier())])),
    ('XGBoost', Pipeline([('scaler', StandardScaler()), ('model', XGBClassifier())])),
    ('SVM', Pipeline([('scaler', StandardScaler()), ('model', SVC())])),
    ('GBM', Pipeline([('scaler', StandardScaler()), ('model', GradientBoostingClassifier())])),
]

#K-nearest neighbors (KNN)
#Classification And Regression Trees (CART)
#Support Vector Machine (SVM)
# Random Forests (RF) 
#Gradient Boosting Machine (GBM)
#xgbost
#Lightgbm

# Entraîner et évaluer chaque modèle du pipeline
for name, pipeline in pipelines:
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("{} - Précision du modèle : {:.2f}%".format(name, accuracy * 100))
    f1 = f1_score(y_test, y_pred)
    print("{} - F1-score du modèle : {:.2f}%".format(name, f1 * 100))
    recall = recall_score(y_test, y_pred)
    print("{} - Recall du modèle : {:.2f}%".format(name, recall * 100))
    roc_auc = roc_auc_score(y_test, y_pred)
    print("{} - ROC AUC du modèle : {:.2f}%".format(name, roc_auc * 100))
    print("\n\n\n")


CART - Précision du modèle : 99.69%
CART - F1-score du modèle : 85.81%
CART - Recall du modèle : 89.66%
CART - ROC AUC du modèle : 94.73%




RF - Précision du modèle : 99.81%
RF - F1-score du modèle : 90.46%
RF - Recall du modèle : 88.28%
RF - ROC AUC du modèle : 94.10%






C:\Users\Admin\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


KNN - Précision du modèle : 99.45%
KNN - F1-score du modèle : 70.77%
KNN - Recall du modèle : 63.45%
KNN - ROC AUC du modèle : 81.64%




LightGBM - Précision du modèle : 99.78%
LightGBM - F1-score du modèle : 89.29%
LightGBM - Recall du modèle : 86.21%
LightGBM - ROC AUC du modèle : 93.07%




XGBoost - Précision du modèle : 99.78%
XGBoost - F1-score du modèle : 89.51%
XGBoost - Recall du modèle : 88.28%
XGBoost - ROC AUC du modèle : 94.09%




SVM - Précision du modèle : 99.05%
SVM - F1-score du modèle : 19.51%
SVM - Recall du modèle : 11.03%
SVM - ROC AUC du modèle : 55.51%




GBM - Précision du modèle : 99.35%
GBM - F1-score du modèle : 57.28%
GBM - Recall du modèle : 42.07%
GBM - ROC AUC du modèle : 71.01%




